In [ ]:
import os
# insert token
# hf_token = os.getenv("HF_TOKEN")
assert hf_token, "HF_TOKEN is not set"

# Authenticate to access Google Cloud resources
from google.colab import auth
auth.authenticate_user()


In [ ]:
!apt-get install -y python3-dev
!pip install --no-binary :all: crcmod

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp311-cp311-linux_x86_64.whl size=31658 sha256=43a34b56d31267cf01452ffcb15533388c3095d49d976f7d64c06d3b2ebf9932
  Stored in directory: /root/.cache/pip/wheels/23/94/7a/8cb7d14597e6395ce969933f01aed9ea8fa5f5b4d4c8a61e99
Successfully built crcmod


In [7]:

from google.colab import auth
auth.authenticate_user()

from google.cloud import storage
from pathlib import Path
import os

In [11]:
import os
from pathlib import Path
import torch
import timm
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import subprocess

split = 'val'

# Set paths
gcs_bucket = "bracs-dataset-bucket"
tile_root_gcs = f"gs://{gcs_bucket}/Tiles/{split}"
embedding_root_gcs = f"gs://{gcs_bucket}/Embeddings/{split}"
tile_root_local = Path(f"/content/tiles/{split}")
embedding_root_local = Path(f"/content/embeddings/{split}")

tile_root_local.mkdir(parents=True, exist_ok=True)
embedding_root_local.mkdir(parents=True, exist_ok=True)

# Load model
tile_encoder = timm.create_model("hf_hub:prov-gigapath/prov-gigapath", pretrained=True).cuda().eval()
print("✅ Tile Encoder loaded.")
print("🧮 Total parameters:", sum(p.numel() for p in tile_encoder.parameters()))

# Image transform
transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])


✅ Tile Encoder loaded.
🧮 Total parameters: 1134953984


In [14]:
def encode_slide_tiles(slide_id: str):
    slide_tile_dir = tile_root_local / slide_id
    save_path = embedding_root_local / slide_id / f"{slide_id}_embeddings.pt"
    save_path.parent.mkdir(parents=True, exist_ok=True)

    image_paths = sorted([p for p in slide_tile_dir.glob("*.png") if p.name != "thumbnail.png"])
    print(f"Found {len(image_paths)} tiles for {slide_id}")

    all_embeddings = []
    all_coords = []

    for img_path in tqdm(image_paths, desc=f"🧠 Encoding {slide_id}"):
        img = Image.open(img_path).convert("RGB")
        x = transform(img).unsqueeze(0).cuda()

        with torch.no_grad():
            embed = tile_encoder(x).squeeze()  # Keep on GPU

        # Extract (x, y) from filename: "x1234_y5678.png"
        name_parts = img_path.stem.split("_")
        x_coord = int(name_parts[0][1:])
        y_coord = int(name_parts[1][1:])

        all_embeddings.append(embed)
        all_coords.append([x_coord, y_coord])

    # Stack on GPU, then move to CPU once
    embeddings_tensor = torch.stack(all_embeddings).cpu()
    coords_tensor = torch.tensor(all_coords)

    num_tiles = len(all_embeddings)
    print(f"📊 {slide_id}: embeddings shape {embeddings_tensor.shape}, coords shape {coords_tensor.shape}")
    print(f"🧩 {slide_id}: total {num_tiles} tile embeddings stacked.")

    torch.save({
        "embeddings": embeddings_tensor,
        "coords": coords_tensor
    }, save_path)

    print(f"✅ Saved slide embeddings to {save_path}")


In [15]:
def run_encoder_pipeline_from_gcs():
    # List all subfolders in the GCS bucket
    list_command = f"gsutil ls {tile_root_gcs}/"
    result = subprocess.run(list_command.split(), capture_output=True, text=True)
    slide_dirs = [line.strip().rstrip("/") for line in result.stdout.strip().split("\n")]

    for slide_path in slide_dirs:
        slide_id = Path(slide_path).name
        print(f"\n🔽 Downloading tiles for {slide_id}...")

        # Download to /content/tiles/{split}/BRACS_xxx
        local_slide_dir = tile_root_local / slide_id
        subprocess.run([
    "gsutil", "-m", "cp", "-r", str(slide_path), str(local_slide_dir.parent)
], check=True)

        # Run inference
        encode_slide_tiles(slide_id)

        # Upload embeddings
        gcs_target = f"{embedding_root_gcs}/{slide_id}"
        subprocess.run(["gsutil", "-m", "cp", "-r", str(embedding_root_local / slide_id), gcs_target], check=True)
        print(f"☁️ Uploaded embeddings for {slide_id} to {gcs_target}")

        # Cleanup
        subprocess.run(["rm", "-rf", str(local_slide_dir)])
        subprocess.run(["rm", "-rf", str(embedding_root_local / slide_id)])


In [ ]:
run_encoder_pipeline_from_gcs()



🔽 Downloading tiles for BRACS_1003660...
Found 1 tiles for BRACS_1003660


🧠 Encoding BRACS_1003660: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


📊 BRACS_1003660: embeddings shape torch.Size([1, 1536]), coords shape torch.Size([1, 2])
🧩 BRACS_1003660: total 1 tile embeddings stacked.
✅ Saved slide embeddings to /content/embeddings/val/BRACS_1003660/BRACS_1003660_embeddings.pt
☁️ Uploaded embeddings for BRACS_1003660 to gs://bracs-dataset-bucket/Embeddings/val/BRACS_1003660

🔽 Downloading tiles for BRACS_1003661...
Found 282 tiles for BRACS_1003661


🧠 Encoding BRACS_1003661: 100%|██████████| 282/282 [00:11<00:00, 23.57it/s]


📊 BRACS_1003661: embeddings shape torch.Size([282, 1536]), coords shape torch.Size([282, 2])
🧩 BRACS_1003661: total 282 tile embeddings stacked.
✅ Saved slide embeddings to /content/embeddings/val/BRACS_1003661/BRACS_1003661_embeddings.pt
☁️ Uploaded embeddings for BRACS_1003661 to gs://bracs-dataset-bucket/Embeddings/val/BRACS_1003661

🔽 Downloading tiles for BRACS_1261...
Found 3334 tiles for BRACS_1261


...

🧠 Encoding BRACS_1368: 100%|██████████| 2345/2345 [01:39<00:00, 23.58it/s]


📊 BRACS_1368: embeddings shape torch.Size([2345, 1536]), coords shape torch.Size([2345, 2])
🧩 BRACS_1368: total 2345 tile embeddings stacked.
✅ Saved slide embeddings to /content/embeddings/val/BRACS_1368/BRACS_1368_embeddings.pt
☁️ Uploaded embeddings for BRACS_1368 to gs://bracs-dataset-bucket/Embeddings/val/BRACS_1368

🔽 Downloading tiles for BRACS_1392...
Found 3724 tiles for BRACS_1392


🧠 Encoding BRACS_1392: 100%|██████████| 3724/3724 [02:37<00:00, 23.58it/s]


📊 BRACS_1392: embeddings shape torch.Size([3724, 1536]), coords shape torch.Size([3724, 2])
🧩 BRACS_1392: total 3724 tile embeddings stacked.
✅ Saved slide embeddings to /content/embeddings/val/BRACS_1392/BRACS_1392_embeddings.pt
☁️ Uploaded embeddings for BRACS_1392 to gs://bracs-dataset-bucket/Embeddings/val/BRACS_1392

🔽 Downloading tiles for BRACS_1393...
Found 3548 tiles for BRACS_1393


🧠 Encoding BRACS_1393:  40%|███▉      | 1407/3548 [00:59<01:31, 23.50it/s]